# Projet Pub ou Pas Pub

## Imports librairies

In [1]:
import matplotlib.pyplot as plt
import ultralytics, os, yaml, time, cv2, requests
from ultralytics import YOLO
ultralytics.checks()

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

Ultralytics YOLOv8.0.53  Python-3.9.16 torch-1.13.1+cpu CPU
Setup complete  (16 CPUs, 15.8 GB RAM, 124.0/465.0 GB disk)


## Récupération flux image pour entrainement

In [ ]:

# for i in range(200):
cap = cv2.VideoCapture('http://192.168.20.37:88/cgi-bin/CGIProxy.fcgi?cmd=snapPicture2&usr=Camera_IA-Greta&pwd=IA-Greta25')
_, frame = cap.read()    
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  
plt.imsave('lol.png', frame)

In [ ]:
for image in os.listdir('datasets/images'):
    img = cv2.imread(f'datasets/images/{image}')
    img = img[:, 420:-420]
    plt.imsave(f'datasets/images/{image}', img)

## Entrainement Yolo

### Création ichier YAML

In [ ]:
cell_yaml = dict(
    train = r'C:\Users\antoi\Documents\GitHub\antoine.lecroart\Concevoir_developper_et_peupler_une_BDD\Utiliser_une_BDD_dans_un_projet\Projet_pub_ou_pas_pub\datasets\images\train',
    val = r'C:\Users\antoi\Documents\GitHub\antoine.lecroart\Concevoir_developper_et_peupler_une_BDD\Utiliser_une_BDD_dans_un_projet\Projet_pub_ou_pas_pub\datasets\images\val',
    nc = 2,
    names = ['01', '02']
)

with open('Pub.yaml', 'w') as outfile:
    yaml.dump(cell_yaml, outfile, default_flow_style=True)

In [ ]:
model_name = 'yolov8n.pt'

model = YOLO(model_name)

In [ ]:
model.train(data = 'Pub.yaml', 
            epochs = 10,
            imgsz = 640, 
            dropout = 0.2,
            project = model_name[:-3], 
            device = 'cpu',
            verbose = True)

## Test Yolo

In [ ]:
# Chemins du modèle entraîné et des images de test
best_path = model_name[:-3] + '/train/weights/best.pt'
source = r'C:\Users\antoi\Documents\GitHub\antoine.lecroart\Concevoir_developper_et_peupler_une_BDD\Utiliser_une_BDD_dans_un_projet\Projet_pub_ou_pas_pub\datasets\images\test'

In [ ]:
# Chargement du modèle
model_trained = YOLO(best_path)

In [ ]:
# Intervalle de confiance (de 0 à 1)
confiance = 0.05

# Prédiction
results = model_trained(source, conf = confiance)

for i, result in enumerate(results):
    plt.imsave(f'results/image{i}.png', result.plot())

# if len(results[0]) == 0:
#     print('coupe le son')

# else:
#     print('laisse le son')


## Affichage raspi

In [13]:
model_name = 'yolov8n.pt'
best_path = model_name[:-3] + '/train/weights/best.pt'
source = 'datasets/images/test'

In [14]:
model_trained = YOLO(best_path)

In [18]:
Pub = True

while True:
    cap = cv2.VideoCapture('http://192.168.20.37:88/cgi-bin/CGIProxy.fcgi?cmd=snapPicture2&usr=Camera_IA-Greta&pwd=IA-Greta25')
    _, frame = cap.read()
    frame = frame[:, 420:-420]
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  
    results = model_trained(frame, conf = 0.5)

    if len(results[0]) != 0 and Pub:
        requests.get(f"http://192.168.20.27/modulesGrove.php?texte=Pas Pub&couleur=0,255,0")
        Pub = False

    if len(results[0]) == 0 and not Pub:
        requests.get(f"http://192.168.20.27/modulesGrove.php?texte=Pub&couleur=255,0,0")
        Pub = True

    time.sleep(1)


0: 640x640 (no detections), 115.5ms
Speed: 1.0ms preprocess, 115.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 105.7ms
Speed: 1.0ms preprocess, 105.7ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 104.7ms
Speed: 2.0ms preprocess, 104.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 113.4ms
Speed: 1.0ms preprocess, 113.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


KeyboardInterrupt: 